[![Notebook Tutorial](__code/__all/notebook_tutorial.png)](https://neutronimaging.ornl.gov/tutorial/notebooks/timepix3_event_nexus)

<img src='__docs/__all/notebook_rules.png' />

# Select Your IPTS 

In [ ]:
from __code import system
from __code.timepix3_event_nexus.timepix3_event_nexus import Timepix3EventNexus

import h5py

system.System.select_working_dir(facility='SNS', instrument='SNAP')
from __code.__all import custom_style
custom_style.style()

import matplotlib.pyplot as plt
%matplotlib notebook

## Prepare UI engine 

In [ ]:
# %gui qt

# Select Event NeXus

In [ ]:
# DEBUGGING ONLY

# He3
#working_dir = "/Users/j35/SNS/SNAP/IPTS-30008/nexus/"
#working_dir = "/Volumes/JeanHardDrive/SNS/SNAP/IPTS-30008/nexus/"
nexus_filename = 'SNAP_58700.nxs.h5'

# MCP
working_dir = "/Users/j35/SNS/SNAP/IPTS-30008/nexus/"
nexus_filename = "run58704_events.h5"
nexus_filename = "run58575_events.h5"

o_event = Timepix3EventNexus(working_dir=system.System.get_working_dir())
#o_event = Timepix3EventNexus(working_dir=working_dir)
#o_event.select_nexus()

import os
nexus_file = os.path.join(working_dir, nexus_filename)
assert os.path.exists(nexus_file)
o_event.load_nexus(nexus_file_name=nexus_file)
o_event.rebin_and_display_data()



In [ ]:
with h5py.File(nexus_file, 'r') as f:
    print(f['events'].keys())

In [ ]:
print(len(o_event.x_array))
print(len(o_even
          t.y_array))
print(len(o_event.tof_ns_array))

In [ ]:
print(f"{np.max(o_event.y_array) =}")
print(f"{np.max(o_event.x_array) =}")

In [ ]:
import numpy as np

working_limit = -1

tof_ns_array = o_event.tof_ns_array[0:working_limit]
x_array = o_event.x_array[0:working_limit]
y_array = o_event.y_array[0:working_limit]

nbr_bin = 10000

#indexes with NaN
index_nan = np.where(np.isnan(tof_ns_array))

# remove those x, y and tof_nx
tof_ns_array_cleaned = np.delete(tof_ns_array, index_nan)
x_array_cleaned = np.delete(x_array, index_nan)
y_array_cleaned = np.delete(y_array, index_nan)

histo_tof, bins_tof = np.histogram(tof_ns_array_cleaned, nbr_bin)

In [ ]:
# going one by one and finding in which bin they belong

In [ ]:
stack_images = np.zeros((nbr_bin, 1024, 1024))
np.shape(stack_images)

In [ ]:
for _index, _tof in enumerate(tof_ns_array):
    _array_location = np.where((bins_tof - _tof) < 0)
    try:
        _location = _array_location[0][-1]
    except IndexError:
        _location = 0
    x = round(x_array[_index])
    y = round(y_array[_index])
    if (x<0) or (y<0) or (x>1023) or (y>1023):
        continue
        
    stack_images[_location, y, x] += 1
#     print(f"{_location =}, {y =}, {x =}")
        

Integrated image

In [ ]:
integrated_image = np.mean(stack_images, axis=0)

fig, ax = plt.subplots(figsize=(5,5),
                      nrows=1, ncols=1)
pt = ax.imshow(integrated_imagea)
plt.colorbar(pt)

In [ ]:
fig, ax = plt.subplots(figsize=(5,5),
                      nrows=1, ncols=1)
pt = ax.imshow(stack_images[100])
plt.colorbar(pt)

In [ ]:
integration = np.sum(stack_images, axis=1)
integration = np.sum(integration, axis=1)

In [ ]:
np.shape(integration)

In [ ]:
fig1, ax1 = plt.subplots(figsize=(5,5),
                      nrows=1, ncols=1,
                        num=nexus_filename)
ax1.plot(bins_tof[:-1], integration)
ax1.set_xlabel("TOF (micros)")
ax1.set_ylabel("Mean counts of each bin")

# export the data inoto NeXus

In [ ]:
working_dir = "/Users/j35/SNS/SNAP/IPTS-30008/nexus/"
nexus_filename = "run58575_histo.h5"
output_nexus_filename = os.path.join(working_dir, nexus_filename)
assert os.path.exists(output_nexus_filename)

In [ ]:
with h5py.File(output_nexus_filename, mode='w') as f:
    f.create_group('entry/histo')
    f.create_dataset('entry/histo/stack', data=stack_images)
    f.create_dataset('entry/histo/number_of_bins', data=nbr_bin)
    f.create_group('entry/infos')
    f.create_dataset('entry/infos/input_nexus_filename', data="run58744_events.h5")
